---
title: "Matrix Factorization for Recommender Systems"
author: "chris"
date: 2024-01-25
draft: false
---

It'd be pretty difficult to have escaped recommender systems in 2024 unless you live somewhere without internet (and in which case you wouldn't be reading this anyways).  We get recommended content from streaming services like Netflix and products from online shopping platforms like Amazon.  Built-in ads recommend products even when we're not trying to shop for anything.  Even if you don't know the term recommender systems, there's a good chance you understand what it does.

I first heard about recommender systems early in my data science journey and in the context of the Netflix prize.

https://en.wikipedia.org/wiki/Netflix_Prize

The netflix prize was a competition for the best collaboratize filtering algorithm to predict user ratings for movies.

A brief history of recommender systems https://arxiv.org/abs/2209.01860 suggests that some of the earliest collaborative filtering algorithms were nearest neighbor techniques followed by matrix factorization, the later of which was used extensively during the Netflix Prize...

https://en.wikipedia.org/wiki/Matrix_factorization_(recommender_systems)

mention content based filtering (using user features in a classification or regression model to predict what they might like)

The winners of the prize proved that matrix factorization models (type of colaborative filtering) were far superior.

in my research of matrix factorization methods, probabilistic matrix factorization (PMF) came up as a seminal improvement over previous MF techniques

https://proceedings.neurips.cc/paper_files/paper/2007/file/d7322ed717dedf1eb4e6e52a37ea7bcd-Paper.pdf

have not used MF for machine learning...

came up even in the PyMC package..

https://www.pymc.io/projects/examples/en/latest/case_studies/probabilistic_matrix_factorization.html#id4

sparse matrices don't allow for SVD methods... regularization penalties aren't practical for large datasets (i.e. millions of observations)... PMF scales linearly with the number of observations..


some intuition...

imagine you have m users and some number of n movies.  you can represent this with a an (m x n) matrix and the values of the matrix indicate the ranking a person gave to the movie.  for simplicity, let's say the value is 1 if they watched it and hit thumbs up, otherwise it's a zero.

of course, not everyone is going to watch every single movie available so there will be missing values in the matrix.  but in order to keep users on the platform, we should be able to recommend good movies they haven't watched otherwise they'll go somewhere else.

before getting to the idea behind matrix factorization, let's say that each user can be described by the vector of movies they liked.  this is similar to how words represented by an embedding vector.  if certain people have similar preferences, then their vectors will be similar (measured by cosine similarity for example).

ok so, users can be described by their vector of movie preferences.  but because there is some similarity in movies, we could probably condense this vector representation into a lower dimensional space (ie << n) and still approximately describe the same person.  that lower dimensional space can be thought of as movie basis vectors.

in the same way we can describe people by the movies they like, we can also describe movies by the people that like them.  usually I'm against stereotypes but I'll go ahead and say some people are just similar (fix).  so we can also condense the movie representation into a smaller space of people basis vectors.  this sounds a lot like SVD/PCA...

back to this (m x n) matrix of users and movies.  just from a dimensionality perspective, it's clear that the product of two smaller matrices (say m x d and d x n) could be used to represent this larger m x n matrix.  using these two smaller matrices make it easier for us to make educated guesses about the vectors that describe users or movies since they have a smaller dimensionality (d << m and d << n).  it's important to note that the product of these matrices is just an approximation, so there is some error involved.  but we can minimize this error in an optimization process similar to gradient descent.

so at a high level, that's exactly what we're doing here with matrix factorization.  the benefit of this is that once we've reached a good guess for (call it matrix U and matrix V) the matrices, then we can use those to impute/predict values for the original matrix.  If an imputed value is a 1 (ie they will probably thumbs up it), then we can recommend it to that user.

show formula for approximation matrix, etc.  derivation of why MAP is better than MLE (sparsity)?


and now for some notation...

we have $n$ users and $m$ movies and a ratings matrix $R \in \mathbb{R}^{m \times n}$ such that $R_{i,j}$ is the rating by user $i$ for movie $j$.  the user latent matrix is $U \in \mathbb{R}^{d \times n}$ and movie latent matrix $V \in \mathbb{R}^{d \times m}$ where $d$ is the latent space dimension and a tunable hyperparameter.

because $R_{i,j}$ is approximated by the product of latent vectors $U_i$ and $V_j$, it is normally distributed with that mean and a specified variance

$$R_{i,j} \sim N(U_i^{\intercal} V_j, \sigma ^ 2)$$

the latent vectors $U_i$ and $V_j$ are also normally distributed with zero mean and but no covariance between features (dimensionality $d$)

$$U_i \sim N(0, \sigma_u ^ 2 \mathbf{I})$$

$$V_j \sim N(0, \sigma_v ^ 2 \mathbf{I})$$

the likelihood function

$$
p(R|U,V,\sigma^2) = \prod\limits_{i}^{n}\prod\limits_{j}^{m} p(R_{i,j} | U_i, V_j)
$$

and prior probabilities (latent matrices)

$$
p(U|\sigma_u^2) = \prod\limits_{i}^{n} p(U_i) \; , \quad p(V|\sigma_v^2) = \prod\limits_j^m p(V_i)
$$

whose product define the posterior distribution.  the MAP solution for this involves taking the log probability of the posterior (see paper) which ultimately results in the following objective function that we want to minimize

$$
E = \frac{1}{2} \sum_i^n \sum_j^m I_{i,j}(R_{i,j} - U_i^\intercal V_j)^2 + \frac{\lambda_u}{2} \sum_i^n \Vert U_i \Vert ^ 2 +  \frac{\lambda_v}{2} \sum_j^m \Vert V_j \Vert ^ 2
$$

where $I_{i,j}$ is an indicator function if user $i$ has rated movie $j$, $\lambda_u = \frac{\sigma^2}{\sigma_u^2}$ and $\lambda_v = \frac{\sigma^2}{\sigma_v^2}$.

The paper defines this objective function but doesn't go into much detail about the optimization procedure.  Because there are two unknown matrices, we need to perform an alternating gradient descent (also known as alternating least squares).  This involves holding either of the latent matrices constant while the other is updated.  See here and here for more detail.

https://engineering.fb.com/2015/06/02/core-infra/recommending-items-to-more-than-a-billion-people/

https://developers.google.com/machine-learning/recommendation/collaborative/matrix

To find the update rules, we need to find partial derivatives for $U_i$ and $V_j$ with respect to the objective function defined in the paper.  So time to dive into some matrix calculus.  Below is my derivation for $\frac{\partial E}{\partial U_i}$ ... note that my notation is pretty loose

$$
\frac{\partial}{\partial U_i} \biggl[ \frac{1}{2}\sum_i^n \sum_j^m I_{i,j}(R_{i,j} - U_i^\intercal V_j)^2 \biggr] + \frac{\partial}{\partial U_i} \biggl[ \frac{\lambda_u}{2} \sum_i^n \Vert U_i \Vert ^ 2 \biggr] + \frac{\partial}{\partial U_i} \biggl[ \frac{\lambda_v}{2} \sum_j^m \Vert V_j \Vert ^ 2 \biggr]
$$

summation over $i$ dissappears as there is only one term that is not a constant when differentiating with respect to $U_i$ and we set to zero since we are trying to minimize a convex function

$$
\lambda_u U_i - \sum_j^m I_{i,j}(R_{i,j} - U_i^\intercal V_j) V_j = 0
$$

$$
\lambda_u U_i - \sum_j^m I_{i,j} R_{i,j} V_j + U_i \sum_j^m I_{i,j} V_j V_j^\intercal = 0
$$

solving for $U_i$

$$
U_i \biggl( \lambda_u \mathbf{I} + \sum_j^m I_{i,j} V_j V_j^\intercal \biggr) = \sum_j^m I_{i,j} R_{i,j} V_j
$$

$$
U_i = \sum_j^m I_{i,j} R_{i,j} V_j \biggl( \lambda_u \mathbf{I} + \sum_j^m I_{i,j} V_j V_j^\intercal \biggr)^{-1}
$$

and it follows similarly for $V_j$

$$
V_j = \sum_i^n I_{i,j} R_{i,j} U_i \biggl(\lambda_v \mathbf{I} + \sum_i^n I_{i,j} U_i U_i^\intercal \biggr)^{-1}
$$

now we can perform alternating least squares optimization which entails updating one variable while holding the other constant and vice-versa...

- add algo/code for ALS optim
- use the logistic function (i.e. sigmoid) to squash preds to [0, 1]
 - map preds to the ratings range and round to nearest
- test on the movie lens dataset (or something smaller) and compare eval metrics with SOTA ?
